In [40]:
# import code block
from selenium import webdriver
from bs4 import BeautifulSoup
import zipfile
import pandas as pd
import csv
import time
import random
import matplotlib.pyplot as plt
from urllib.request import urlopen
import re
import string
import numpy as np
import sys
# !{sys.executable} -m pip --upgrade pip
# !{sys.executable} -m pip install descartes
!{sys.executable} -m pip install geopandas
# import descartes
import geopandas as gpd
# from shapely.geometry import Point, Polygon
# from shapely import wkt
from datetime import date
import datetime

In [13]:
ERMA = pd.read_csv('RAFT_ERMA_Mergedv3.csv')
ERMA = ERMA.drop('Unnamed: 0', 1)
ERMA

/Users/gonzo/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Tenant ID,Check Date,Actual Individual Payment,Unit ZIP,PAY_TYPE,CETRACT,Key Tenant Key
0,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7284.0,NaN
1,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7301.0,NaN
2,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7283.0,NaN
3,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7291.0,NaN
4,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7307.0,NaN
...,...,...,...,...,...,...,...
750240,T97072,03/15/2021,-500,1841.0,RAFT,2512.0,97072.0
750241,T97072,03/15/2021,-500,1841.0,RAFT,2511.0,97072.0
750242,T97072,03/15/2021,-500,1841.0,RAFT,2510.0,97072.0
750243,T97072,03/15/2021,-500,1841.0,RAFT,2509.0,97072.0


In [16]:
evictionLab = pd.read_csv('tracts.csv')
evictionLab.rename(columns={'name':'CETRACT'}, inplace=True)
evictionLab

,GEOID,year,CETRACT,parent-location,population,poverty-rate,renter-occupied-households,pct-renter-occupied,median-gross-rent,median-household-income,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,25001010100,2000,101.0,"Barnstable County, Massachusetts",3433.09,16.28,849,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
1,25001010100,2001,101.0,"Barnstable County, Massachusetts",3433.09,16.28,842,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
2,25001010100,2002,101.0,"Barnstable County, Massachusetts",3433.09,16.28,836,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
3,25001010100,2003,101.0,"Barnstable County, Massachusetts",3433.09,16.28,829,46.2,644.01,32722.44,...,0.0,2.88,0.82,31.0,30.0,3.62,3.74,0,0,0
4,25001010100,2004,101.0,"Barnstable County, Massachusetts",3433.09,16.28,822,46.2,644.01,32722.44,...,0.0,2.88,0.82,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25136,25027761400,2012,7614.0,"Worcester County, Massachusetts",6567.00,4.19,174,8.4,1333.00,126061.00,...,0.0,2.50,0.00,1.0,1.0,0.57,0.57,1,0,0
25137,25027761400,2013,7614.0,"Worcester County, Massachusetts",6567.00,4.19,180,8.4,1333.00,126061.00,...,0.0,2.50,0.00,4.0,4.0,2.23,2.23,1,0,0
25138,25027761400,2014,7614.0,"Worcester County, Massachusetts",6567.00,4.19,185,8.4,1333.00,126061.00,...,0.0,2.50,0.00,1.0,1.0,0.54,0.54,1,0,0
25139,25027761400,2015,7614.0,"Worcester County, Massachusetts",6567.00,4.19,191,8.4,1333.00,126061.00,...,0.0,2.50,0.00,0.0,0.0,0.00,0.00,0,0,0


In [38]:
# merging eviction lab data
mergedData = ERMA.merge(evictionLab,on='CETRACT',how='inner').drop_duplicates()
mergedData

,Tenant ID,Check Date,Actual Individual Payment,Unit ZIP,PAY_TYPE,CETRACT,Key Tenant Key,GEOID,year,parent-location,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7284.0,NaN,25027728400,2000,"Worcester County, Massachusetts",...,0.02,0.28,0.00,NaN,NaN,NaN,NaN,0,0,0
1,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7284.0,NaN,25027728400,2001,"Worcester County, Massachusetts",...,0.02,0.28,0.00,6.0,5.0,2.74,3.29,0,0,0
2,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7284.0,NaN,25027728400,2002,"Worcester County, Massachusetts",...,0.02,0.28,0.00,2.0,2.0,1.10,1.10,1,0,0
3,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7284.0,NaN,25027728400,2003,"Worcester County, Massachusetts",...,0.02,0.28,0.00,2.0,2.0,1.10,1.10,1,1,0
4,T82460,7/24/2020,"$2,857.37",1520.0,ERMA,7284.0,NaN,25027728400,2004,"Worcester County, Massachusetts",...,0.02,0.28,0.00,5.0,5.0,2.74,2.74,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12842857,T80783,03/01/2021,706,2568.0,RAFT,2001.0,80783.0,25007200100,2012,"Dukes County, Massachusetts",...,0.00,1.65,2.12,15.0,11.0,1.52,2.07,0,0,0
12842858,T80783,03/01/2021,706,2568.0,RAFT,2001.0,80783.0,25007200100,2013,"Dukes County, Massachusetts",...,0.00,1.65,2.12,21.0,18.0,2.42,2.82,0,0,0
12842859,T80783,03/01/2021,706,2568.0,RAFT,2001.0,80783.0,25007200100,2014,"Dukes County, Massachusetts",...,0.00,1.65,2.12,18.0,12.0,1.58,2.36,0,0,0
12842860,T80783,03/01/2021,706,2568.0,RAFT,2001.0,80783.0,25007200100,2015,"Dukes County, Massachusetts",...,0.00,1.65,2.12,12.0,9.0,1.15,1.54,0,0,0


In [39]:
def zipConverter(val):
    result = str(val)
    result = result.split('.')[0]
    if len(result) == 4:
        result = '0' + result
    # print(result)
    return str(result)

mergedData['Unit ZIP'] = mergedData['Unit ZIP'].apply(zipConverter)

,Tenant ID,Check Date,Actual Individual Payment,Unit ZIP,PAY_TYPE,CETRACT,Key Tenant Key,GEOID,year,parent-location,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,T82460,7/24/2020,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2000,"Worcester County, Massachusetts",...,0.02,0.28,0.00,NaN,NaN,NaN,NaN,0,0,0
1,T82460,7/24/2020,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2001,"Worcester County, Massachusetts",...,0.02,0.28,0.00,6.0,5.0,2.74,3.29,0,0,0
2,T82460,7/24/2020,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2002,"Worcester County, Massachusetts",...,0.02,0.28,0.00,2.0,2.0,1.10,1.10,1,0,0
3,T82460,7/24/2020,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2003,"Worcester County, Massachusetts",...,0.02,0.28,0.00,2.0,2.0,1.10,1.10,1,1,0
4,T82460,7/24/2020,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2004,"Worcester County, Massachusetts",...,0.02,0.28,0.00,5.0,5.0,2.74,2.74,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12842857,T80783,03/01/2021,706,02568,RAFT,2001.0,80783.0,25007200100,2012,"Dukes County, Massachusetts",...,0.00,1.65,2.12,15.0,11.0,1.52,2.07,0,0,0
12842858,T80783,03/01/2021,706,02568,RAFT,2001.0,80783.0,25007200100,2013,"Dukes County, Massachusetts",...,0.00,1.65,2.12,21.0,18.0,2.42,2.82,0,0,0
12842859,T80783,03/01/2021,706,02568,RAFT,2001.0,80783.0,25007200100,2014,"Dukes County, Massachusetts",...,0.00,1.65,2.12,18.0,12.0,1.58,2.36,0,0,0
12842860,T80783,03/01/2021,706,02568,RAFT,2001.0,80783.0,25007200100,2015,"Dukes County, Massachusetts",...,0.00,1.65,2.12,12.0,9.0,1.15,1.54,0,0,0


In [42]:
def dateConverter(val):
    vals = val.split('/')
    month = vals[0]
    if len(month) == 1:
        month = ('0' + month)
    day = vals[1]
    if len(day) == 1:
        day = '0' + day 
    year = vals[2]
    result = year + '-' + month + '-' + day
    return date.fromisoformat(result)

mergedData['Check Date'] = mergedData['Check Date'].apply(dateConverter)

In [43]:
mergedData

,Tenant ID,Check Date,Actual Individual Payment,Unit ZIP,PAY_TYPE,CETRACT,Key Tenant Key,GEOID,year,parent-location,...,pct-nh-pi,pct-multiple,pct-other,eviction-filings,evictions,eviction-rate,eviction-filing-rate,low-flag,imputed,subbed
0,T82460,2020-07-24,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2000,"Worcester County, Massachusetts",...,0.02,0.28,0.00,NaN,NaN,NaN,NaN,0,0,0
1,T82460,2020-07-24,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2001,"Worcester County, Massachusetts",...,0.02,0.28,0.00,6.0,5.0,2.74,3.29,0,0,0
2,T82460,2020-07-24,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2002,"Worcester County, Massachusetts",...,0.02,0.28,0.00,2.0,2.0,1.10,1.10,1,0,0
3,T82460,2020-07-24,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2003,"Worcester County, Massachusetts",...,0.02,0.28,0.00,2.0,2.0,1.10,1.10,1,1,0
4,T82460,2020-07-24,"$2,857.37",01520,ERMA,7284.0,NaN,25027728400,2004,"Worcester County, Massachusetts",...,0.02,0.28,0.00,5.0,5.0,2.74,2.74,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12842857,T80783,2021-03-01,706,02568,RAFT,2001.0,80783.0,25007200100,2012,"Dukes County, Massachusetts",...,0.00,1.65,2.12,15.0,11.0,1.52,2.07,0,0,0
12842858,T80783,2021-03-01,706,02568,RAFT,2001.0,80783.0,25007200100,2013,"Dukes County, Massachusetts",...,0.00,1.65,2.12,21.0,18.0,2.42,2.82,0,0,0
12842859,T80783,2021-03-01,706,02568,RAFT,2001.0,80783.0,25007200100,2014,"Dukes County, Massachusetts",...,0.00,1.65,2.12,18.0,12.0,1.58,2.36,0,0,0
12842860,T80783,2021-03-01,706,02568,RAFT,2001.0,80783.0,25007200100,2015,"Dukes County, Massachusetts",...,0.00,1.65,2.12,12.0,9.0,1.15,1.54,0,0,0
